# Portfolio Optimization
Creating two investment portfolios of $10,000 with four stocks, where the first portfolio will have equal weighting among the stocks, and the second portfolio will be optimized for risk-adjusted return. 
1. The process includes importing two years of stock data 
2. building the initial portfolio with equal weighting, analyzing and visualizing it 
3. generating 10,000 portfolio scenarios with random weightings
4. identifying the best portfolio among the scenarios while visualizing the results.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
stock_list = ['AMD', 'AAPL', 'MSFT', 'ORCL']

stocks = {}

for i_stock in stock_list:
    stocks[i_stock] = pd.read_csv(str(i_stock + '.csv'), parse_dates=True, index_col = 'Date')

In [ ]:
stocks['AMD'].head()

## Equal-Weighted Portfolio
* Normalized Return = Adjusted Close / Adjusted Close on the `startdate` of the portfolio
* Allocation = Normalized Return * 0.25 (equal weighting for each of the four stocks)
* Position Value = Allocation * 10,000 (value of the portfolio)

In [ ]:
for stock_name, stock_data in stocks.items():
    first_adj_close = stock_data.iloc[0]['Adj Close']
    stock_data['Normalized Return'] = stock_data['Adj Close'] / first_adj_close

In [ ]:
stocks['AAPL'].head()

In [ ]:
for stock_name, stock_data in stocks.items():
    stock_data['Allocation'] = stock_data['Normalized Return'] * 0.25

In [ ]:
stocks['MSFT'].head()

In [ ]:
for stock_name, stock_data in stocks.items():
    stock_data['Position Value'] = stock_data['Allocation'] * 10000

In [ ]:
stocks['ORCL'].head()

## Visualize Portfolio Performance

In [ ]:
position_values = {}

for stock_name, stock_data in stocks.items():
    position_values[stock_name] = stock_data['Position Value']

In [ ]:
position_values = pd.DataFrame(data=position_values)

position_values.head()

In [ ]:
position_values['Total'] = position_values.sum(axis=1)

In [ ]:
position_values.head()

In [ ]:
plt.figure(figsize=(12, 8))

plt.plot(position_values['Total'])

plt.title('Equal-Weighted Portfolio Performance')
plt.ylabel('Total Value');

In [2]:
plt.figure(figsize=(12, 8))

plt.plot(position_values.iloc[:,0:4])

plt.title('Equal-Weighted Portfolio Stock Performance')
plt.ylabel('Total Value');

NameError: name 'plt' is not defined

## Performance Metrics for the Portfolio
 * Cumulative Return
 * Mean Daily Return
 * Standard Deviation Daily Return
 * Sharpe Ratio
 * Annualized Sharpe Ratio

In [ ]:
end_value = position_values['Total'][-1]
start_value = position_values['Total'][0]

cumulative_return = end_value / start_value - 1

print(str(round(cumulative_return*100,2)), '%')

In [ ]:
position_values['Daily Return'] = position_values['Total'].pct_change()

position_values.head()

In [ ]:
mean_daily_return = position_values['Daily Return'].mean()

print('The mean daily return is:', str(round(mean_daily_return, 4)))

In [ ]:
std_daily_return = position_values['Daily Return'].std()

print('The std daily return is:', str(round(std_daily_return, 4)))

### Sharpe Ratio
The Sharpe ratio is a useful metric for quantifying the amount of return generated by a given level of risk. When comparing two different investments against the same benchmark, the investment with a higher Sharpe ratio offers a greater return for the same level of risk or the same return for a lower risk than the other investment. The Sharpe ratio formula involves subtracting a risk-free rate (e.g., government bond yields) from the average return of the portfolio and then dividing the result by the standard deviation of the return. Since we assume the risk-free rate to be negligible, we can omit it from the Sharpe ratio calculation in this case.




In [ ]:
sharpe_ratio = mean_daily_return / std_daily_return

sharpe_ratio

In [ ]:
sharpe_ratio_annualized = sharpe_ratio * 252**0.5

sharpe_ratio_annualized

## Scenarios to Optimize Portfolio Weighting
Generating the scenarios to optimize the portfolio weighting:
 * Dictionary containing the adjusted close for each of our stocks: stock_adj_close
 * Dictionary that transforms the adjusted close for each day to a percent change from the previous day


In [ ]:
stock_adj_close = {}

for stock_name, stock_data in stocks.items():
    stock_adj_close[stock_name] = stock_data['Adj Close']

In [ ]:
stock_adj_close = pd.DataFrame(data=stock_adj_close)

stock_adj_close.head()

In [ ]:
stock_returns = stock_adj_close.pct_change()

stock_returns.head()

## Build & Run 10,000 Portfolio Scenarios
B uild the structures required to generate these scenarios and store the output. To do this, will use the `numpy.zeros()` function. 
 * weights_array - this array will have 10,000 rows and 4 columns and hold the weighting allocation for each stock
 * returns_array - this array will contain the portfolio return for each scenario
 * volatility_array - this array will contain the portfolio volatility for each scenario
 * sharpe_array - this array will contain the sharpe ratio for each scenario

In [ ]:
scenarios = 10000

weights_array = np.zeros((scenarios, len(stock_returns.columns)))

weights_array

In [ ]:
returns_array = np.zeros(scenarios)
volatility_array = np.zeros(scenarios)
sharpe_array = np.zeros(scenarios)

In [ ]:
import random
random.seed(3)
np.random.seed(3)

for index in range(scenarios): 
    numbers = np.array(np.random.random(4))
    
    weights = numbers / np.sum(numbers)
    
    weights_array[index,:] = weights
    
    returns_array[index] = np.sum(stock_returns.mean()*252*weights)
    
    volatility_array[index] = np.sqrt(np.dot(weights.T,np.dot(stock_returns.cov()*252, weights)))

    sharpe_array[index] = returns_array[index] / volatility_array[index]

In [ ]:
print("The first combination:", weights_array[0])

In [ ]:
print("The sharpe ratio of the first portfolio:", sharpe_array[0])

## Identify the Optimal Portfolio


In [ ]:
sharpe_array.max()

In [ ]:
index_max_sharpe = sharpe_array.argmax()
index_max_sharpe

In [ ]:
print(stock_list)
print(weights_array[index_max_sharpe,:])

## Visualize the Optimal Portfolio & Portfolio Scenarios

In [ ]:
plt.figure(figsize=(12,8))

plt.scatter(volatility_array, returns_array, c=sharpe_array, cmap='viridis')

plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return');

In [3]:
max_sharpe_return = returns_array[index_max_sharpe]
max_sharpe_volatility = volatility_array[index_max_sharpe]

plt.figure(figsize=(12,8))

plt.scatter(volatility_array, returns_array, c=sharpe_array, cmap='viridis')

plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')

plt.scatter(max_sharpe_volatility, max_sharpe_return, c='orange', edgecolors='black');

NameError: name 'returns_array' is not defined